In [21]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [22]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2020-10-24 01:22:35,-23.12,-134.97,74.03,81,100,14.99
1,1,Dingle,PH,2020-10-24 01:22:35,11.00,122.67,75.87,92,100,10.00
2,2,Port Alfred,ZA,2020-10-24 01:22:35,-33.59,26.89,68.45,89,99,17.76
3,3,Barrow,US,2020-10-24 01:20:02,71.29,-156.79,28.40,92,90,10.29
4,4,Bathsheba,BB,2020-10-24 01:22:35,13.22,-59.52,84.20,70,40,24.16


In [23]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [24]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [25]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [26]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [27]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [29]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2020-10-24 01:22:35,-23.12,-134.97,74.03,81,100,14.99
1,1,Dingle,PH,2020-10-24 01:22:35,11.00,122.67,75.87,92,100,10.00
4,4,Bathsheba,BB,2020-10-24 01:22:35,13.22,-59.52,84.20,70,40,24.16
5,5,Namatanai,PG,2020-10-24 01:21:21,-3.67,152.43,86.32,59,15,2.21
10,10,Kattivakkam,IN,2020-10-24 01:22:36,13.22,80.32,78.80,83,75,9.17
13,13,Kavieng,PG,2020-10-24 01:22:37,-2.57,150.80,84.61,62,84,3.98
15,15,Cabo San Lucas,MX,2020-10-24 01:18:01,22.89,-109.91,86.00,58,5,10.29
16,16,Hobyo,SO,2020-10-24 01:22:37,5.35,48.53,81.07,77,100,11.01
18,18,Kapaa,US,2020-10-24 01:22:38,22.08,-159.32,82.00,78,90,10.29
22,22,Jamestown,US,2020-10-24 01:22:38,42.10,-79.24,70.00,60,75,12.75


In [30]:
preferred_cities_df.count()

City_ID       231
City          231
Country       228
Date          231
Lat           231
Lng           231
Max Temp      231
Humidity      231
Cloudiness    231
Wind Speed    231
dtype: int64

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,74.03,-23.12,-134.97,
1,Dingle,PH,75.87,11.00,122.67,
4,Bathsheba,BB,84.20,13.22,-59.52,
5,Namatanai,PG,86.32,-3.67,152.43,
10,Kattivakkam,IN,78.80,13.22,80.32,
13,Kavieng,PG,84.61,-2.57,150.80,
15,Cabo San Lucas,MX,86.00,22.89,-109.91,
16,Hobyo,SO,81.07,5.35,48.53,
18,Kapaa,US,82.00,22.08,-159.32,
22,Jamestown,US,70.00,42.10,-79.24,


In [45]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [46]:
 hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,74.03,-23.12,-134.97,Pension Maro'i
1,Dingle,PH,75.87,11.00,122.67,Camp Pasica
4,Bathsheba,BB,84.20,13.22,-59.52,Atlantis Hotel
5,Namatanai,PG,86.32,-3.67,152.43,Namatanai Hotel
10,Kattivakkam,IN,78.80,13.22,80.32,St. Joseph Thirumana mandapam
...,...,...,...,...,...,...
576,Tanuku,IN,74.16,16.75,81.70,Hotel Sri Devi Residency
577,Naze,JP,71.60,28.37,129.48,奄美ポートタワーホテル
581,Lakshmeshwar,IN,71.60,15.13,75.47,helpclues
589,Nioaque,BR,72.81,-21.14,-55.83,Nioaque Novo Hotel


In [35]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [39]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [40]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))